In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps
import json
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
covid_csv = "Resources/owid-covid-data.csv"
covid_df = pd.read_csv(covid_csv)
covid_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,ZWE,Africa,Zimbabwe,2021-09-09,126056.0,125.0,134.000,4521.0,4.0,10.286,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116543,ZWE,Africa,Zimbabwe,2021-09-10,126163.0,107.0,118.857,4532.0,11.0,10.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116544,ZWE,Africa,Zimbabwe,2021-09-11,126220.0,57.0,113.571,4536.0,4.0,10.000,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
116545,ZWE,Africa,Zimbabwe,2021-09-12,126269.0,49.0,102.714,4538.0,2.0,8.000,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


In [3]:
# group the data set by location and find the max for the total cases up-to-date
group_covid = covid_df.groupby(["location"])["total_cases"].max()
group_covid

location
Afghanistan             154094.0
Africa                 8057274.0
Albania                 157436.0
Algeria                 200301.0
Andorra                  15096.0
                        ...     
Wallis and Futuna            NaN
World                225244735.0
Yemen                     8452.0
Zambia                  207960.0
Zimbabwe                126399.0
Name: total_cases, Length: 233, dtype: float64

In [4]:
# drop NAN values to clean the dataset
clean_covid = group_covid.dropna(inplace=False)
clean_covid_df = pd.DataFrame(clean_covid).reset_index()
clean_covid_df

,location,total_cases
0,Afghanistan,154094.0
1,Africa,8057274.0
2,Albania,157436.0
3,Algeria,200301.0
4,Andorra,15096.0
...,...,...
197,Vietnam,624547.0
198,World,225244735.0
199,Yemen,8452.0
200,Zambia,207960.0


In [5]:
# Check the max value for the dataset
max_cases = clean_covid_df.loc[clean_covid_df["total_cases"] == clean_covid_df["total_cases"].max()]
max_cases
# # as the max is for the whole world, drop this one
# clean_covid_df_location_only = clean_covid_df.drop(index=max_cases.index)
# clean_covid_df_location_only

# max_cases111 = clean_covid_df_location_only.loc[clean_covid_df_location_only["total_cases"] == clean_covid_df_location_only["total_cases"].max()]
# max_cases111

,location,total_cases
198,World,225244735.0


In [6]:
location_list = clean_covid_df["location"].to_list()
location_list

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'European Union',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bi

In [7]:
# Too many locations, call API spend too many time, only call when the test code work well!

# # Retriving lat and lng for locations
# for location in location_list:
#     # Build the endpoint URL
#     target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_key}"


#     # Run a request to endpoint and convert result to json
#     geo_data = requests.get(target_url).json()
#     # Print the json
#     pprint(geo_data)
#     try:
#         # Extract latitude and longitude for the locations in dataset
#         lat = geo_data["results"][0]["geometry"]["location"]["lat"]
#         clean_covid_df.loc["lat"] = lat
#         lng = geo_data["results"][0]["geometry"]["location"]["lng"]
#         clean_covid_df.loc["lng"] = lng
#     except:
#         print("Cannot find the location")

In [9]:
test = clean_covid_df.head()
test

# Retriving lat and lng for locations
for index,row in test.iterrows():
    location = row["location"]
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_key}"


    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    # Print the json
    pprint(geo_data)
    try:
        # Extract latitude and longitude for the locations in dataset
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        test.loc[index,"Lat"] = lat
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
        test.loc[index,"Lng"] = lng
    except:
        print("Cannot find the location")

{'results': [{'address_components': [{'long_name': 'Afghanistan',
                                      'short_name': 'AF',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Afghanistan',
              'geometry': {'bounds': {'northeast': {'lat': 38.49087670000001,
                                                    'lng': 74.8898619},
                                      'southwest': {'lat': 29.3772,
                                                    'lng': 60.5170005}},
                           'location': {'lat': 33.93911, 'lng': 67.709953},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 38.49087670000001,
                                                      'lng': 74.8898619},
                                        'southwest': {'lat': 29.3772,
                                                      'lng': 60.5170005}}},
              'place_id':

E:\ANACONDA3\envs\pythonData\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
E:\ANACONDA3\envs\pythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


{'results': [{'address_components': [{'long_name': 'Africa',
                                      'short_name': 'Africa',
                                      'types': ['continent',
                                                'establishment',
                                                'natural_feature']}],
              'formatted_address': 'Africa',
              'geometry': {'bounds': {'northeast': {'lat': 37.5359,
                                                    'lng': 63.80859390000001},
                                      'southwest': {'lat': -47.1313489,
                                                    'lng': -25.383911}},
                           'location': {'lat': -8.783195, 'lng': 34.508523},
                           'location_type': 'APPROXIMATE',
                           'viewport': {'northeast': {'lat': 33.8352384,
                                                      'lng': 59.31900329999999},
                                        'southwest': {

In [10]:
test

,location,total_cases,Lat,Lng
0,Afghanistan,154094.0,33.939110,67.709953
1,Africa,8057274.0,-8.783195,34.508523
2,Albania,157436.0,41.153332,20.168331
3,Algeria,200301.0,28.033886,1.659626
4,Andorra,15096.0,42.506285,1.521801


In [14]:
# Store latitude and longitude in locations, store total cases data in covid_total_cases
locations = test[["Lat", "Lng"]]
covid_total_cases = test["total_cases"]
covid_total_cases

0     154094.0
1    8057274.0
2     157436.0
3     200301.0
4      15096.0
Name: total_cases, dtype: float64

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
# The highest humidity for this dataframe is 100, set max_intensity to 100
heat_layer = gmaps.heatmap_layer(locations, weights=covid_total_cases, 
                                 dissipating=False, max_intensity=test["total_cases"].max(),
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))